Here we implement the solver from [Quantum algorithm for solving linear differential equations: Theory and experiment](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307) by Xin *et al.* in the **Classiq** quantum programming language. 

### Classical Problem

Xin *et al.* provide a new method to solve any Linear Differential Equation formulated as 

$$\frac{d}{dt}\vec{x}(t)= M\vec{x}(t)+\vec{b}$$

for an $N\times N$ matrix $M$ and $N$-dimensional vectors $x$, $b$. 

Taylor-expanding the analytical solution, 

$$
\begin{aligned}
x(t)&=e^{Mt}x(0)+\left(e^{Mt}-\mathbb{I}\right)M^{-1}b
\\
&\approx \sum^k_{m=0}\frac{(Mt)^m}{m!}x(0)+\sum^k_{n=1}\frac{M^{n-1}t^n}{n!}b.
\end{aligned}
$$

### Quantum Formulation

1. Prepare two quantum states $\ket{x(0)}$ and $\ket{b}$ in the $N$-dimensional computational basis, on $\log_2(N)$ qubits. 
1. Describe the $N\times N$ operator $M$ as $A$ in the computational basis, such that $A_{ij}=M_{ij}\ket{i}\bra{j}$ up to normalization.

We then arrive at 

$$\ket{x(t)}\approx \sum_{m=0}^k \frac{\|x(0)\|(\|M\|At)^m}{m!}\ket{x(0)}+\sum_{n=1}^k\frac{\|b\|(\|M\|A)^{n-1}t^n}{n!}\ket{b}$$


#### More Encodings
1. Assuming $A$ is unitary, encode $k$ powers of $A$ as the unitaries $U_n=A^n$.
1. Define $C_m=\|x(0)\|(\|M\|t)^m/m!$ and $D_n=\|b\|(\|M\|t)^{n-1}t/n!$



In [4]:
import numpy as np
import math
from util import make_quantum, is_unitary
from Code_For_V_and_W_Operators import V_operator, normalization, construct_VS1_VS2, construct_W_WS1_WS2

x0 = make_quantum([1,1])
b = make_quantum([0,0])
# taylor approximation
k = 3
# want to repeat QCircuit for discrete times
t = 0.5

nbits = len(x0)
nqubits = np.ceil(np.log2(nbits))


M = [[0,1],[-1,0]]
M = M / np.linalg.norm(M, ord='fro')

C, D, Cvals, Dvals = normalization(M, k, x0, b, t)
N = np.sqrt(C**2 + D**2)
V = V_operator(C, D)
VS1, VS2 = construct_VS1_VS2(Cvals, Dvals, C, D, k)
W, WS1, WS2 = construct_W_WS1_WS2(V, VS1, VS2)

# jth element of x(t) is N^2 <j|x(t)> 


In [ ]:
from classiq import (
    qfunc, Output, QBit, QNum, QArray, CInt,
    allocate, within_apply, control, repeat, if_, 
    inplace_prepare_state, hadamard_transform, inplace_prepare_int, 
    create_model, write_qmod,
    X, H, SWAP, 
    Constraints, Preferences,
)
from classiq.synthesis import synthesize, set_constraints, set_execution_preferences, set_preferences
from classiq.executor import execute
from classiq.interface.ide.show import show
from classiq.interface.generator.quantum_program import QuantumProgram



def prepare_registers(*args):
    """ for each pair of args (register, values), 
    use inplace_prepare_state to prepare the register with the values """
    for i in range(0, len(args), 2):
        inplace_prepare_state(args[i+1], args[i])

# define the main program that will be compiled
# into a .qmod quantum circuit file
@qfunc
def main(work_register: Output[QArray], x0_b_ancilla: Output[QBit], 
         taylor_register: Output[QNum]):
    # encode x_0 and b into "work" register using ancilla qubit
    allocate(1, x0_b_ancilla)
    allocate(nqubits, work_register)
    H(x0_b_ancilla)
    # encode x_0 and C with 0
    control(x0_b_ancilla == 0, lambda: prepare_registers(work_register, b, taylor_register, Dvals))
    # encode b and D with 1
    control(x0_b_ancilla == 1, 
            lambda: prepare_registers(work_register, b, taylor_register, Dvals))

    # use log_2(k+1) qubits to store the taylor approximation


